In [25]:
import numpy as np
from matplotlib import pyplot as pl
from scipy.integrate import quad

# Problem 1

In [103]:
def rect(f, a, b, n):
    tot, err = 0, 0
    dx = (b - a)/n
    x = a
    next_val = f(a)
    val = 0
    while x < b:
        val = next_val
        tot += val * dx
        x += dx
        next_val = f(x)
        err += dx * abs(val-next_val) / 2

    return tot, err, dx

def rect_vec(f, a, b, n):
    xs = np.linspace(a, b, n+1)[:-1]
    dx = (xs[1]-xs[0])
    vals = np.vectorize(f)(xs)
    err = 0.5 * np.sum(np.abs(vals[1:]-vals[:-1])) * dx
    return np.sum(vals)*dx, err


def trap(f, a, b, n):
    tot, err = 0, 0
    dx = (b - a)/n
    x = a + dx/2
    next_val = f(x)
    err = abs(next_val - f(a)) / 2 * dx
    val = 0
    while x < b:
        val = next_val
        tot += val * dx
        x += dx
        next_val = f(x)
    return tot, err

In [16]:
f = lambda x: x**(-3/2)
a, b = 1, 5

In [105]:
#%timeit t1, e1, step = rect(f, a, b, 100)
#%timeit t2, e2 = rect_vec(f, a, b, 100)
#%timeit tt, et = trap(f, a, b, 100)
#%timeit q, qe, *_ = quad(f, a, b)
print(t1, e1)
print(t2, e2)
print(tt, et)
print(q, qe)

1.1239803303199585 0.01821114561800018
1.12398033031996 0.018189462681181722
1.105474638616042 0.0005853422945750131
1.1055728090000843 5.444307667957722e-13


In [29]:
2 * (1 - 5**-0.5)

1.1055728090000843

In [104]:
tt, et = trap(f, a, b, 100)